# Lab05: TF-IDF.

- MSSV: 1712746
- Họ và tên: Nguyễn Minh Tâm

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `tf_idf_data.txt` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập và biểu diễn dữ liệu bằng không gian vector và trọng số TF-IDF: https://en.wikipedia.org/wiki/Web_mining.

## Nội dung bài tập

## 1. Không gian vector

- Một vector 2 chiều có thể được viết dưới dạng `[x,y]`
- Một vector 3 chiều có thể được viết dưới dạng `[x,y,z]`
![vector](vector.png)

- Đặt $V$ là tập hợp các đặc trưng trong thể hiện dữ liệu.
- Bất kỳ mẫu dữ liệu nào đều có thể được biểu diễn dưới dạng một vectơ với $\vert V\vert$ chiều
- Ví dụ: giả sử chúng ta có 3 đặc trưng là các từ dog, bite, man. Giá trị 1 thể hiện từ đó xuất hiện 1 lần, 0 là không xuất hiện.  
![space](space.png)


## 2. TF-IDF (Term Frequency - Inverse Document Frequency)
TF-IDF cho biết độ quan trọng của một từ đối với một tài liệu trong ngữ liệu.

TF- Term Frequency : dùng để ước lượng tần xuất xuất hiện của từ trong văn bản. Tuy nhiên với mỗi văn bản thì có độ dài khác nhau số lần xuất hiện của từ có thể nhiều hơn . Vì vậy số lần xuất hiện của từ sẽ được chuẩn hóa bằng cách chia cho độ dài của văn bản (tổng số từ trong văn bản đó).
    
$$tf_{t}=\dfrac{f(t,d)}{\sum_{i \in d}f(i,d)}$$ 

- $f(t,d)$ là số lần xuất hiện từ $t$ trong văn bản $d$.

IDF- Inverse Document Frequency: Khi tính tần số xuất hiện TF thì các từ đều được coi là quan trọng như nhau. Tuy nhiên có một số từ thường được được sử dụng nhiều nhưng không quan trọng để thể hiện ý nghĩa của đoạn văn như "is", "the"... (các từ chức năng). Ta cần giảm độ quan trọng của những từ này.

$$idf_t=\log \left(\dfrac{n}{df_t}\right)$$

- *n* là số văn bản.

- $df_t$ là số văn bản xuất hiện từ t

**TF-IDF:** $$tf_{t} \times idf_t$$

- $tf_{t}$ càng lớn tần suất xuất hiện của từ trong văn bản càng lớn.
- $idf_t$ càng lớn từ hiếm khi xuất hiện trong tập dữ liệu.
- **Giả định** những đặc trưng quan trọng nhất là những đặc trưng hiếm xuất hiện.

## 3. Thu thập và thể hiện dữ liệu
Thu thập dữ liệu bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining.
- Gọi $D$ là một tập văn bản chứa *n* văn bản: $D=\left\{d_1,d_2,...,d_n\right\}$.
- $V=\left\{v_1,v_2,...v_{\vert V \vert}\right\}$ là từ điển (tất cả các từ phân biệt trong dữ liệu thu được). $\vert V \vert$ là kích thước từ điển.
- Một trọng số $w_{ij}$ được gán cho từ $t_i$ của văn bản dj thuộc $D$ xác định mức quan trọng của $t_i$ trong văn bản $d_j$. Từ không xuất hiện trong $d_j$ có $w_{ij}=0$.
- Mỗi văn bản $d_j$ được thể hiện dưới dạng vector $\mathbf{d_j}= [w_{1j},w_{2j},...,w_{\vert V \vert j}]$
- Thể hiện dữ liệu bằng một ma trận M kích thước $n \times \vert V \vert$ => mỗi hàng thể hiện một văn bản.

In [17]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle
# addition
import numpy as np
import sys
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import math

def drawProgressBar(percent, barLen = 20): # show percent complete https://stackoverflow.com/questions/3002085/python-to-print-out-status-bar-and-percentage
    sys.stdout.write("\r")
    progress = ""
    for i in range(barLen):
        if i < int(barLen * percent):
            progress += "="
        else:
            progress += " "
    sys.stdout.write("[ %s ] %.2f%%" % (progress, percent * 100))
    sys.stdout.flush()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mtosity\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def valid_url(url): 
    # django url validation regex https://stackoverflow.com/questions/7160737/how-to-validate-a-url-in-python-malformed-or-not
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return (re.match(regex, url) is not None)


def get_urls(url):
    success = False
    html_context = ""
    try:
        r=requests.get(url)
        success = True
        if(r.status_code != 200 or ("text/html" not in r.headers["content-type"])):
           return []
        html_context = r.content
    except:
        success = False
        return []
    # TODO
    # Lấy các url nằm trong trang web của url này, lưu lại vào biến urls
    urls = []
    soup = BeautifulSoup(html_context, "lxml") # lxml is just the parser for reading the html
    for a in soup.find_all('a', href=True):
        if(valid_url(a['href'])): # check url, vì có khi là chỉ là url header hay sub url hay url image/pdf
            urls.append(a['href'])
    return urls

def get_urls_recursive(start_url, limit):
    urls = [start_url]
    counter = 1
    visited = {} # dict {url: True,...}
    for url in urls:
        if(counter >= limit):
            return urls
        # TODO
        # Lấy các url nằm trong trang web của url này, lưu lại vào biến new_urls
        # Với mỗi url mới trong new_urls:
        #   Nếu nó chưa nằm trong urls thì thêm nó vô  
        # Nếu kích thước của urls vượt quá limit thì dừng và xóa phần dư thừa
        new_urls = get_urls(url)
        for new_url in new_urls:
            if(counter >= limit):
                return urls
            if(not hasattr(visited, new_url)):
                visited[new_url] = True
                urls.append(new_url)
                counter += 1
        
    return urls
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

def wordList(url):
    success = False
    html_context = ""
    try:
        r=requests.get(url)
        success = True
        if(r.status_code != 200 or ("text/html" not in r.headers["content-type"])):
           return []
        html_context = r.content
    except:
        success = False
        return []
        
    soup=BeautifulSoup(html_context,"html.parser")
    text=soup.findAll(text=True)
    filtered_text=list(filter(text_filter,text))
    word_list=[]
    #TODO:
    str_punc = string.punctuation
    str_split = ''
    for idx, c in enumerate(str_punc):
        if(idx == 0):
            str_split += f"{c}"
        else:
            str_split += f"|{c}"
    
    for sentence in filtered_text:
        if(sentence not in str_punc):
            next_words = re.sub('['+string.punctuation+']', '', sentence).split() #https://www.geeksforgeeks.org/python-extract-words-from-given-string/
            word_list = word_list + next_words
    return word_list

def read_url(url, url_idx, data):
    word_list=wordList(url)
    
    #TODO
    '''Initialize value: data[word] = [[url], 1]'''
                                    #list_url
    #VD: data.get("at")==None => initialize data["at"]
        # data.get("at")!=None >= add url_idx to list_url_idx (remember to check if this url_idx is in list_url_idx), 
                                # frequence+=1
    for word in word_list:
        if(data.get(word)==None):
            data[word] = [[url_idx], 1]
        else:
            # update index urls
            if url_idx not in data[word][0]:
                data[word][0].append(url_idx)
            # update count
            data[word][1] += 1
    return True

def read_url_v2(url, url_idx, data):
    word_list=wordList(url)
    
    #TODO
    '''Initialize value: data[word] = [[url], [1]]'''
                                    #list_url
    #VD: data.get("at")==None => initialize data["at"]
        # data.get("at")!=None >= add url_idx to list_url_idx (remember to check if this url_idx is in list_url_idx), 
                                # frequence+=1
    for word in word_list:
        if(data.get(word)==None):
            data[word] = [[url_idx], [1]]
        else:
            try:
                data_word_url_index = data[word][0].index(url_idx)
                data[word][1][data_word_url_index] += 1
            except:
                data[word][0].append(url_idx)
                data[word][1].append(1)
    return len(word_list)

In [3]:
n_urls=200
print("Getting urls");
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', n_urls)
data = {} # {word: [[url_idx]: [count]]}
num_words = [] # number of words for each url

print("Count words from urls");
for url_index, url in enumerate(url_list, 1):
    new_num_words = read_url_v2(url, url_index, data)
    num_words.append(new_num_words)
    drawProgressBar((url_index+1) / len(url_list)) # just show complete percentage
    
print("Removing stop words")
english_stopwords = stopwords.words('english')
for stopword in english_stopwords:
    data.pop(stopword, None)
    
n_words = len(data)
list_words = list(data.keys())
print("Done")

Getting urls
Count words from urls
[ ==================== ] 100.50%Removing stop words
Done


In [34]:
M=np.zeros(shape=(n_urls,n_words),dtype=float)

In [35]:
for word_index, word in enumerate(list_words):
    word_counting = data[word]
    url_indexes = word_counting[0];
    counts = word_counting[1];
    for i in range(len(url_indexes)):
        url_index = url_indexes[i];
        count = counts[i];
        M[url_index][word_index] += count
        
print('Done converting data to M')

Done converting data to M


In [36]:
print('checking shape:')
print(M.shape)
print(len(num_words)) # num_words[url_index] = count 
print(len(list_words)) 

checking shape:
(200, 15189)
200
15189


In [39]:
# TF
for i in range(len(num_words)):
    if(num_words[i] != 0):
        M[i] = M[i] / num_words[i]
    
# IDF
for j in range(len(list_words)):
    n = n_urls
    word = list_words[j]
    dft = len(data[word][0])
    if(dft == 0):
        print(word)
    M[:, j] = M[:, j] * math.log(n / dft)

In [42]:
M

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.54018549e-02, 2.22881596e-02, 4.78912893e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.79665260e-02, 3.42894764e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [5.26430959e-03, 1.23711577e-03, 0.00000000e+00, ...,
        2.06803957e-03, 2.06803957e-03, 2.06803957e-03],
       [1.18683220e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.65605091e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [44]:
with open("tf_idf_data.txt", "wb") as f:
    np.save(f, M)